<a href="https://colab.research.google.com/github/aditipriya25/ISRO/blob/main/NeRF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class NeRF(nn.Module):
    def __init__(self, D=8, W=256):
        super(NeRF, self).__init__()
        self.D = D
        self.W = W
        self.layers = nn.ModuleList([nn.Linear(3, W)] +
                                    [nn.Linear(W, W) if i != 4 else nn.Linear(W + 3, W) for i in range(D - 1)])
        self.fc_alpha = nn.Linear(W, 1)
        self.fc_rgb = nn.Linear(W, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        h = x
        for i, layer in enumerate(self.layers):
            h = self.relu(layer(h))
            if i == 4:
                h = torch.cat([h, x], -1)
        alpha = self.fc_alpha(h)
        rgb = self.fc_rgb(h)
        return torch.cat([rgb, alpha], -1)


In [3]:
# Assume you have rays_o, rays_d, and target_srgb for training
# rays_o: [N, 3] ray origins
# rays_d: [N, 3] ray directions
# target_srgb: [N, 3] ground truth RGB values

rays_o = ...  # Load or generate ray origins
rays_d = ...  # Load or generate ray directions
target_srgb = ...  # Load target RGB values


In [4]:
from google.colab import files

# Upload the .img file
uploaded = files.upload()


Saving M104318871LC_pyr.tif to M104318871LC_pyr (1).tif


In [5]:
# Install Miniconda
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Initialize Conda
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
!conda init bash

# Install ISIS3
!conda install -c usgs-astrogeology isis3 -y


--2024-06-25 18:13:09--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143808873 (137M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 137.15M   132MB/s    in 1.0s    

2024-06-25 18:13:10 (132 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [143808873/143808873]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONP

In [6]:
import os
img_file_path = '/content/M104318871LC_pyr.tif'
cub_file_path = 'output_image.cub'

# Convert the .img file to .cub using ISIS3

!m68k2isis from={img_file_path} to={cub_file_path}


/bin/bash: line 1: m68k2isis: command not found


In [7]:
import torch.nn as nn

# ... (rest of your code)

def generate_rays(camera, image_shape):
    # Placeholder function for generating rays
    # Normally, this should be derived from actual camera parameters
    H, W = image_shape[:2]
    camera = {
        'position': [0, 0, 0],
        'orientation': [0, 0, 0],

    }
    rays_o = np.random.rand(H, W, 3)
    rays_d = np.random.rand(H, W, 3)

    return rays_o, rays_d

image_data = np.random.rand(100, 100, 3)
rays_o, rays_d = generate_rays(None, image_data.shape)

# Convert to PyTorch tensors - removing .cuda() calls
rays_o = torch.tensor(rays_o, dtype=torch.float32)
rays_d = torch.tensor(rays_d, dtype=torch.float32)
target_srgb = torch.tensor(image_data, dtype=torch.float32)

print("Data preparation completed")

Data preparation completed


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class NeRF(nn.Module):
    def __init__(self, D=8, W=256):
        super(NeRF, self).__init__()
        self.D = D
        self.W = W
        self.layers = nn.ModuleList([nn.Linear(3, W)] +
                                    [nn.Linear(W, W) if i != 4 else nn.Linear(W + 3, W) for i in range(D - 1)])
        self.fc_alpha = nn.Linear(W, 1)
        self.fc_rgb = nn.Linear(W, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        h = x
        for i, layer in enumerate(self.layers):
            h = self.relu(layer(h))
            if i == 4:
                h = torch.cat([h, x], -1)
        alpha = self.fc_alpha(h)
        rgb = self.fc_rgb(h)
        return torch.cat([rgb, alpha], -1)

# Training loop (simplified)
model = NeRF()
optimizer = optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.MSELoss()
H, W = 128, 128  # Example image dimensions
rays_o = torch.rand(H, W, 3)
rays_d = torch.rand(H, W, 3)
target_srgb = torch.rand(H, W, 3)

for epoch in range(10000):
    t_vals = torch.linspace(0, 1, steps=64)
    points = rays_o.unsqueeze(2) + rays_d.unsqueeze(2) * t_vals.unsqueeze(0).unsqueeze(0).unsqueeze(3)
    points = points.view(-1, 3)

    rgb_alpha = model(points)
    rgb = rgb_alpha[:, :3]
    rgb = rgb.view(target_srgb.shape[0], target_srgb.shape[1], 64, 3).mean(dim=2)

    loss = criterion(rgb, target_srgb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 0.3704618513584137


In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from pathlib import Path

In [ ]:
# Install required packages
!pip uninstall imagecodecs
!pip install imagecodecs
!pip list imagecodecs
import imagecodecs

# Import necessary libraries
import numpy as np
import torch
import tifffile
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS

try:
    # Load.tif image file using tifffile
    data = tifffile.imread("/content/M104318871LC_pyr.tif")
    imgs, poses, focal = data["images"], data["poses"], data["focal"]
    _, H, W, _ = imgs.shape

    # Prepare test image and pose for visualization
    test_img = torch.tensor(imgs[101])
    test_pose = torch.tensor(poses[101])

    # Remove the test image and pose from the dataset
    imgs = np.delete(imgs, 101, axis=0)
    poses = np.delete(poses, 101, axis=0)

    # Print image dimensions
    print(f"H={H}, W={W}")

    # Visualize the test image
    plt.imshow(test_img.numpy())
    plt.show()

    # Create a WCS (World Coordinate System) object
    wcs = WCS(naxis=2)  # 2D image
    wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']  # Set WCS parameters
    wcs.wcs.crval = [0, 0]  # Set WCS reference values
    wcs.wcs.crpix = [0, 0]  # Set WCS reference pixels
    wcs.wcs.cdelt = [1, 1]  # Set WCS pixel scales

    # Set the header with WCS information
    header = wcs.to_header()

    # Create a new FITS file with the ISIS (CUB type image) format
    hdu = fits.PrimaryHDU(imgs, header=header)
    hdu.writeto('output.cub', overwrite=True)

    # Verify the output
    plt.imshow(fits.getdata('output.cub')[0])
    plt.show()

except ValueError as e:
    print(f"Error reading the TIFF file: {e}")

Found existing installation: imagecodecs 2024.6.1
Uninstalling imagecodecs-2024.6.1:
  Would remove:
    /usr/local/bin/imagecodecs
    /usr/local/lib/python3.12/site-packages/imagecodecs-2024.6.1.dist-info/*
    /usr/local/lib/python3.12/site-packages/imagecodecs.libs/libLerc-b0c2f582.so.4
    /usr/local/lib/python3.12/site-packages/imagecodecs.libs/libSPERR-35265dae.so.0.8.1
    /usr/local/lib/python3.12/site-packages/imagecodecs.libs/libaec-98fa4c42.so.0.1.3
    /usr/local/lib/python3.12/site-packages/imagecodecs.libs/libaom-3825d194.so.3.9.0
    /usr/local/lib/python3.12/site-packages/imagecodecs.libs/libavif-61b03158.so.16.0.4
    /usr/local/lib/python3.12/site-packages/imagecodecs.libs/libblosc-750cd085.so.1.21.5
    /usr/local/lib/python3.12/site-packages/imagecodecs.libs/libblosc2-adcc849a.so.2.14.4
    /usr/local/lib/python3.12/site-packages/imagecodecs.libs/libbrotlicommon-5311ed18.so.1.1.0
    /usr/local/lib/python3.12/site-packages/imagecodecs.libs/libbrotlidec-dd121758.so.